In [1]:
import os
from skimage.metrics import structural_similarity as ssim
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable
# from scipy.optimize import linprog

In [2]:
def eve_distortion(eve_intensities): #from the reconstructed image in uint8 type
    feve_intensities = eve_intensities.astype(np.float)
    fintensities = intensities.astype(np.float)
    dif = (fintensities -  feve_intensities)
    eve_dist_mat = dif**2
    occurance_sum = height*width #?!
    eve_dist = eve_dist_mat.sum()/occurance_sum
    return eve_dist

# Prepare Image Data

In [3]:
image_name = 'house'
img_file = image_name + '.tiff'
intensities = cv2.imread(img_file, cv2.IMREAD_GRAYSCALE)
height = np.shape(intensities)[0]
width = np.shape(intensities)[1]
imf = np.float32(intensities)

'''From scratch'''
all_nq_ac = []
all_dcs = []
for i in range(0,int(height/8)):
    print(i)
    for j in range(0,int(width/8)):
        blk = imf[8*i:8*(i+1), 8*j:8*(j+1)]
        shifted_blk = blk - 128 #level_shift for dct
        dct = cv2.dct(shifted_blk)
        nq_coef = np.round(dct)
        nq_coef_dc = nq_coef[0,0]
        nq_coef_ac = np.delete(nq_coef, [0,0])

        all_dcs = np.append(all_dcs, nq_coef_dc)
        all_nq_ac = np.append(all_nq_ac, nq_coef_ac)


dcs_file_name = image_name + '_dcs.npy'        
np.save(dcs_file_name, all_dcs)        
acs_file_name = image_name + '_acs.npy'        
np.save(acs_file_name, all_nq_ac)
'''...........................................'''

''' If saved already, so comment the dct part'''
# dcs_file_name = image_name + '_dcs.npy' 
# acs_file_name = image_name + '_acs.npy'
# all_dcs = np.load(dcs_file_name)
# all_nq_ac = np.load(acs_file_name)
'''...........................................'''

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63


'...........................................'

In [4]:
ac_values, ac_counts = np.unique(all_nq_ac, return_counts=True)
ac_values_dic = {}
ac_counts_dic = {}
ac_bits_all_sizes = []
for i in range(1,11):
    neg_part = range(-(2**i-1), -2**(i-1)+1)
    pos_part = range(2**(i-1), 2**i)
    values_in_size = np.append(neg_part,pos_part)
    ac_values_dic[i] = values_in_size
    
    counts_in_size = np.zeros(np.size(values_in_size))
    for j in range(np.size(values_in_size)):
        if values_in_size[j] in ac_values:
            position_in_counts = np.where(
                ac_values == values_in_size[j])
            counts_in_size[j] = ac_counts[position_in_counts]
            
    ac_counts_dic[i] = counts_in_size
    
    #Rate
    ac_bits_in_size = sum(counts_in_size)*i
    ac_bits_all_sizes = np.append(ac_bits_all_sizes, ac_bits_in_size)
total_ac_bits = sum(ac_bits_all_sizes)

ac_prbs = ac_counts/sum(ac_counts)


n0 = 0
pos0 = np.where(ac_values == 0)
ac_values_n0 = ac_values.copy()
ac_counts_n0 = ac_counts.copy()
ac_values_n0 = np.delete(ac_values_n0, pos0)
ac_counts_n0 = np.delete(ac_counts_n0, pos0)


# select 'num' from the middle and sides
num = 256
part1 = ac_values_n0[0:int(num/4)]
neg1_ind = np.where(ac_values_n0 == -1)[0]
part2 = ac_values_n0[int(neg1_ind-int(num/4)+1):\
                     int(neg1_ind+int(num/4)+1)]
part3 = np.append(ac_values_n0[-int(num/4):-1], ac_values_n0[-1])
selected_values_1 = np.append(part1, part2)
selected_values = np.append(selected_values_1, part3)

part1_c = ac_counts_n0[0:int(num/4)]
part2_1_c = ac_counts_n0[int(neg1_ind-int(num/4)+1):int(neg1_ind+1)]
part2_2_c = ac_counts_n0[int(neg1_ind+1):int(neg1_ind+int(num/4)+1)]
part2_c = np.append(part2_1_c, part2_2_c)
part3_c = np.append(ac_counts_n0[-int(num/4):-1], ac_counts_n0[-1])
selected_counts_1 = np.append(part1_c, part2_c)
selected_counts = np.append(selected_counts_1, part3_c)

org_ac_values_n0 = ac_values_n0.copy()
org_ac_counts_n0 = ac_counts_n0.copy()

# Taking care of not selected values, their pairs
ns_part1 = ac_values_n0[int(num/4):int(neg1_ind-int(num/4)+1)]
ns_part2 = ac_values_n0[int(neg1_ind+int(num/4)+1):\
                       -int(num/4)]
ns_values = np.append(ns_part1, ns_part2)

ns_pairs_elements1 = []
ns_pairs_elements2 = []
no_pair_elements_neg = []
no_pair_elements_pos = []
for i in ns_part1:
    i_size = np.floor(np.log2(abs(i)))+1
    bs_pair = i+2**i_size-1+2**(i_size-1)
    if bs_pair in ns_part2:
        ns_pairs_elements1 = np.append(ns_pairs_elements1,i)
        ns_pairs_elements2 = np.append(ns_pairs_elements2,bs_pair)
    else:
        no_pair_elements_neg = np.append(no_pair_elements_neg, i)

for i in ns_part2:
    if not i in ns_pairs_elements2:
        no_pair_elements_pos = np.append(no_pair_elements_pos, i)
        
no_pair_min_len = min(len(no_pair_elements_neg),\
                      len(no_pair_elements_pos))
no_pair_elements1 = no_pair_elements_neg[0:no_pair_min_len]
no_pair_elements2 = no_pair_elements_pos[0:no_pair_min_len]

extra_elements_1 =  no_pair_elements_neg[no_pair_min_len:\
                                        len(no_pair_elements_neg)]
extra_elements_2 =  no_pair_elements_pos[no_pair_min_len:\
                                        len(no_pair_elements_pos)]
extra_elements = np.append(extra_elements_1, extra_elements_2)

extra_half_ind = int(np.floor(len(extra_elements)/2))
extra_elements1 = extra_elements[0:extra_half_ind]
# extra_elements2 = extra_elements[extra_half_ind:]
extra_elements2 = extra_elements[extra_half_ind:-1]

sac_values_n0 = selected_values
sac_counts_n0 = selected_counts

# Optimization Parameters

In [5]:
'''This cell takes time'''

var = sac_values_n0.copy()
prbs = sac_counts_n0/sum(sac_counts_n0)

cells_num = np.size(var)

all_yijs = []
all_dijs = []
all_prb0s = []
all_entropies = []
for i in range(0,cells_num-1):
    for j in range(i+1,cells_num):
        yi = var[i]
        yj = var[j]
        possible_dij = 0.5*(
          var[i]**2*prbs[i] + yi**2*prbs[i] - yi*2*var[i]*prbs[i]\
        + var[j]**2*prbs[j] + yi**2*prbs[j] - yi*2*var[j]*prbs[j]\
        + var[i]**2*prbs[i] + yj**2*prbs[i] - yj*2*var[i]*prbs[i]\
        + var[j]**2*prbs[j] + yj**2*prbs[j] - yj*2*var[j]*prbs[j])
        
        all_dijs = np.append(all_dijs, possible_dij)
                
        possible_prb0 = max(prbs[j],prbs[i])
        all_prb0s = np.append(all_prb0s, possible_prb0)
        
        sum_prbs = prbs[j]+prbs[i]
        pair_entropy = -sum_prbs*np.log2(sum_prbs)
        all_entropies = np.append(all_entropies, pair_entropy)

######### ######### ######### ######### ######### ######### ##############        
all_length = int(cells_num * (cells_num-1) / 2)

A_eq = np.zeros([cells_num, all_length])
k = 0
for i in range(0, cells_num-1):
    for j in range(i+1,cells_num):
        A_eq[i,k]=1
        k+=1
k = 0
for j in range(1, cells_num):
    for i in range (j,cells_num):
        A_eq[i,k]=1
        k+=1     

# Creating Model

In [6]:
# Define the model
model = LpProblem(name="pulp_milp", sense=LpMaximize)

# Define the decision variables
x = {i: LpVariable(name=f"x{i}", lowBound=0, cat="Integer") for i in range(1, all_length+1)}

In [7]:
'''DO NOT RUN'''
'''Just this cell takes most of the time'''
# Add constraints
for i in range(0,np.shape(A_eq)[0]):
    model += (lpSum(list(x.values())*A_eq[i,:]) == 1)

# List of Lambdas

In [39]:
Lambdas1 = np.arange(0, 100000, 10000)
Lambdas2 = np.arange(100000, 160000, 5000)
Lambdas3 = np.arange(160000, 192730, 1000)
Lambdas4 = [192730,192731]
Lambdas5 = np.arange(193000, 200000, 500)
Lambdas6 = np.arange(200000, 300000, 5000)
Lambdas7 = np.arange(300000, 700000, 50000)

Lambdas = np.concatenate(
    (Lambdas1, Lambdas2, Lambdas3, Lambdas4,
     Lambdas5, Lambdas6, Lambdas7), axis = 0)
problematic_lambdas = np.array([70000, 80000, 167000, 185000])
Lambdas = np.setdiff1d(Lambdas, problematic_lambdas)
len(Lambdas)

95

In [12]:
Lambdas1 = np.arange(100000, 200000, 10000)
Lambdas = Lambdas1

In [16]:
pairs_all_lambdas = []
mip_pairs_show_dic = {}
opt_pairs_prbs_lambda_dic = {}
eve_msb0_dist_db_dic = {}
eve_msb1_dist_db_dic = {}
optimized_ac_bits_dic = {}
rate_increase_dic = {}
optimized_entropy_dic = {}

avg_psnrs = []
psnr_msb0s = []
psnr_msb1s = []
entropies = []
rate_increases = []
ac_dist_total_dbs = []

# Set the objective
Omega = 1
Lambdas = np.arange(160731,160740,1)
# Lambdas = [161000, 160000, 160000, 160000, 160000, 160000, 160000, 161000]
for Lambda in Lambdas:
    print('lambda=',Lambda)
    mip_pairs_show = []
    model += lpSum((Omega * all_dijs - Lambda * all_entropies\
                   )*list(x.values()))


    # Solve the optimization problem
    status = model.solve()

#     # Get the results
#     print(f"status: {model.status}, {LpStatus[model.status]}")
#     print(f"objective: {model.objective.value()}")


    x01 = [] 
    for var in x.values(): 
        x01 = np.append(x01, var.value())

#     print('check', np.unique(x01))
##################################################################
       
    var = sac_values_n0.copy()

    mip_pairs = x01

    k = 0
    mip_xx1 = np.zeros([cells_num, cells_num])
    for i in range(0,cells_num):
        for j in range(i+1,cells_num):
            mip_xx1[i,j] = mip_pairs[k]
            k +=1


    mip_pairs_show = np.array(np.where(mip_xx1 == 1))

    true_pairs_show_p1 = []
    for i in mip_pairs_show[0]:
        true_pairs_show1 = var[int(i)]
        true_pairs_show_p1 = np.append(true_pairs_show_p1, 
                                       true_pairs_show1)

    true_pairs_show_p2 = []
    for i in mip_pairs_show[1]:
        true_pairs_show1 = var[int(i)]
        true_pairs_show_p2 = np.append(true_pairs_show_p2, 
                                       true_pairs_show1)

    true_pairs_show = [true_pairs_show_p1, true_pairs_show_p2]

    mip_pairs_show = true_pairs_show

    unique_opt_val = np.unique(mip_pairs_show)
#     print('check', np.unique(unique_opt_val - sac_values_n0))
#     print('just selected pairs\n', np.transpose(mip_pairs_show))    ################################################################    
    # Attaching not optimized elements
    # Return to original ac_values_n0 after optimization
    mip_pairs_show[0]= np.append(mip_pairs_show[0], ns_pairs_elements1)
    mip_pairs_show[0]= np.append(mip_pairs_show[0], no_pair_elements1)
    mip_pairs_show[0]= np.append(mip_pairs_show[0], extra_elements1)

    mip_pairs_show[1]= np.append(mip_pairs_show[1], ns_pairs_elements2)
    mip_pairs_show[1]= np.append(mip_pairs_show[1], no_pair_elements1)
    mip_pairs_show[1]= np.append(mip_pairs_show[1], extra_elements2)
    
    if len(mip_pairs_show[0]) != len(mip_pairs_show[1]):
        i = np.argmax([len(mip_pairs_show[0]), len(mip_pairs_show[1])])
        mip_pairs_show[i] = np.delete(mip_pairs_show[i], -1)

    prbs = []
    ac_values_n0 = org_ac_values_n0.copy()
    ac_counts_n0 = org_ac_counts_n0.copy()
    prbs = ac_counts_n0/sum(ac_counts_n0)  
    
#############################################################    
    ##### For Lambdas
    mip_pairs_show_dic[Lambda] = mip_pairs_show

    pairs_show_file_name = image_name + '_pairs_' +str(Lambda)+'.npy'
    np.save(pairs_show_file_name, mip_pairs_show)

lambda= 160731
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/mehr/anaconda3/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/bcd9d957be5841a4bca99965774106ea-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/bcd9d957be5841a4bca99965774106ea-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 261 COLUMNS
At line 163462 RHS
At line 163719 BOUNDS
At line 196360 ENDATA
Problem MODEL has 256 rows, 32640 columns and 65280 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is -624137 - 0.51 seconds
Cgl0004I processed model has 256 rows, 32640 columns (32640 integer (32640 of which binary)) and 65280 elements
Cbc0038I Initial state - 14 integers unsatisfied sum - 7
Cbc0038I Pass   1: suminf.    0.00000 (0) obj. 662265 iterations 547
Cbc0038I Solution found of 662265
Cbc0038I Before mini branch and bound

Cbc0038I Pass  31: suminf.    3.00000 (6) obj. 624226 iterations 161
Cbc0038I Pass  32: suminf.    3.00000 (6) obj. 624226 iterations 348
Cbc0038I Pass  33: suminf.    3.00000 (6) obj. 624226 iterations 13
Cbc0038I Pass  34: suminf.    2.54993 (7) obj. 624271 iterations 386
Cbc0038I Pass  35: suminf.    2.43588 (7) obj. 624271 iterations 462
Cbc0038I Pass  36: suminf.    3.00000 (6) obj. 624234 iterations 263
Cbc0038I Pass  37: suminf.    3.00000 (9) obj. 624271 iterations 100
Cbc0038I Pass  38: suminf.    3.49466 (9) obj. 624271 iterations 110
Cbc0038I Pass  39: suminf.   15.51414 (34) obj. 624271 iterations 379
Cbc0038I No solution found this major pass
Cbc0038I Before mini branch and bound, 32467 integers at bound fixed and 0 continuous
Cbc0038I Full problem 256 rows 32640 columns, reduced to 30 rows 61 columns
Cbc0038I Mini branch and bound improved solution from 624326 to 624326 (4.12 seconds)
Cbc0038I Round again with cutoff of 624232
Cbc0038I Reduced cost fixing fixed 13072 vari

Cbc0038I Full problem 256 rows 32640 columns, reduced to 50 rows 102 columns
Cbc0038I Mini branch and bound improved solution from 624410 to 624405 (3.76 seconds)
Cbc0038I Round again with cutoff of 624293
Cbc0038I Reduced cost fixing fixed 10594 variables on major pass 4
Cbc0038I Pass  34: suminf.    3.00000 (6) obj. 624193 iterations 30
Cbc0038I Pass  35: suminf.    3.00000 (6) obj. 624200 iterations 56
Cbc0038I Pass  36: suminf.    3.00000 (6) obj. 624200 iterations 119
Cbc0038I Pass  37: suminf.    3.00000 (6) obj. 624200 iterations 3
Cbc0038I Pass  38: suminf.    4.77641 (35) obj. 624293 iterations 393
Cbc0038I Pass  39: suminf.    2.97603 (7) obj. 624293 iterations 525
Cbc0038I Pass  40: suminf.    2.91509 (7) obj. 624293 iterations 213
Cbc0038I Pass  41: suminf.    3.00000 (6) obj. 624288 iterations 273
Cbc0038I Pass  42: suminf.    3.00000 (9) obj. 624293 iterations 250
Cbc0038I Pass  43: suminf.    3.00000 (6) obj. 624288 iterations 382
Cbc0038I Pass  44: suminf.    3.98289 (1

Cbc0038I Full problem 256 rows 32640 columns, reduced to 14 rows 25 columns
Cbc0038I Mini branch and bound improved solution from 624414 to 624327 (3.64 seconds)
Cbc0038I Round again with cutoff of 624274
Cbc0038I Reduced cost fixing fixed 11428 variables on major pass 4
Cbc0038I Pass  33: suminf.    3.00000 (6) obj. 624197 iterations 37
Cbc0038I Pass  34: suminf.    3.00000 (9) obj. 624274 iterations 85
Cbc0038I Pass  35: suminf.    3.00000 (9) obj. 624274 iterations 8
Cbc0038I Pass  36: suminf.   17.00000 (37) obj. 624274 iterations 378
Cbc0038I Pass  37: suminf.    3.00000 (9) obj. 624274 iterations 529
Cbc0038I Pass  38: suminf.    3.00000 (9) obj. 624274 iterations 75
Cbc0038I Pass  39: suminf.    3.00000 (9) obj. 624274 iterations 3
Cbc0038I Pass  40: suminf.    3.00000 (6) obj. 624272 iterations 109
Cbc0038I Pass  41: suminf.    3.00000 (9) obj. 624274 iterations 34
Cbc0038I Pass  42: suminf.    3.02514 (9) obj. 624274 iterations 109
Cbc0038I Pass  43: suminf.    3.00000 (6) obj

Cbc0038I Full problem 256 rows 32640 columns, reduced to 58 rows 132 columns
Cbc0038I Mini branch and bound improved solution from 624418 to 624397 (4.32 seconds)
Cbc0038I Round again with cutoff of 624301
Cbc0038I Reduced cost fixing fixed 10606 variables on major pass 4
Cbc0038I Pass  33: suminf.    3.00000 (6) obj. 624201 iterations 23
Cbc0038I Pass  34: suminf.    3.00000 (6) obj. 624201 iterations 31
Cbc0038I Pass  35: suminf.    3.00000 (6) obj. 624201 iterations 45
Cbc0038I Pass  36: suminf.    1.07679 (7) obj. 624301 iterations 155
Cbc0038I Pass  37: suminf.    1.07679 (7) obj. 624301 iterations 177
Cbc0038I Pass  38: suminf.    3.00000 (6) obj. 624201 iterations 156
Cbc0038I Pass  39: suminf.    1.07679 (7) obj. 624301 iterations 169
Cbc0038I Pass  40: suminf.   11.99698 (33) obj. 624301 iterations 257
Cbc0038I Pass  41: suminf.    3.00000 (9) obj. 624301 iterations 795
Cbc0038I Pass  42: suminf.    3.00000 (9) obj. 624301 iterations 18
Cbc0038I Pass  43: suminf.    3.00000 (9

lambda= 160736
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/mehr/anaconda3/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/62fe117ef4ba47b88a7ee24492af9188-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/62fe117ef4ba47b88a7ee24492af9188-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 261 COLUMNS
At line 163462 RHS
At line 163719 BOUNDS
At line 196360 ENDATA
Problem MODEL has 256 rows, 32640 columns and 65280 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is -624157 - 0.36 seconds
Cgl0004I processed model has 256 rows, 32640 columns (32640 integer (32640 of which binary)) and 65280 elements
Cbc0038I Initial state - 14 integers unsatisfied sum - 7
Cbc0038I Pass   1: suminf.    0.00000 (0) obj. 662286 iterations 416
Cbc0038I Solution found of 662286
Cbc0038I Before mini branch and bound

Cbc0038I Pass  82: suminf.    3.00000 (9) obj. 624261 iterations 202
Cbc0038I Pass  83: suminf.    3.00000 (9) obj. 624261 iterations 0
Cbc0038I Pass  84: suminf.    3.00000 (6) obj. 624223 iterations 623
Cbc0038I Pass  85: suminf.    3.00000 (6) obj. 624223 iterations 67
Cbc0038I Pass  86: suminf.    3.00000 (6) obj. 624223 iterations 0
Cbc0038I Pass  87: suminf.    3.00000 (6) obj. 624223 iterations 231
Cbc0038I Pass  88: suminf.    3.00000 (6) obj. 624223 iterations 106
Cbc0038I Pass  89: suminf.    3.00000 (9) obj. 624261 iterations 198
Cbc0038I Pass  90: suminf.    3.00000 (9) obj. 624261 iterations 0
Cbc0038I Pass  91: suminf.    3.00000 (6) obj. 624223 iterations 617
Cbc0038I No solution found this major pass
Cbc0038I Before mini branch and bound, 32596 integers at bound fixed and 0 continuous
Cbc0038I Mini branch and bound did not improve solution (5.52 seconds)
Cbc0038I After 5.52 seconds - Feasibility pump exiting with objective of 624342 - took 4.64 seconds
Cbc0012I Integer 

Cbc0038I Full problem 256 rows 32640 columns, reduced to 42 rows 83 columns
Cbc0038I Mini branch and bound improved solution from 624425 to 624351 (2.93 seconds)
Cbc0038I Round again with cutoff of 624294
Cbc0038I Reduced cost fixing fixed 11116 variables on major pass 4
Cbc0038I Pass  33: suminf.    3.00000 (6) obj. 624209 iterations 27
Cbc0038I Pass  34: suminf.    3.00000 (9) obj. 624294 iterations 62
Cbc0038I Pass  35: suminf.    3.00000 (9) obj. 624294 iterations 2
Cbc0038I Pass  36: suminf.    3.00000 (9) obj. 624294 iterations 90
Cbc0038I Pass  37: suminf.    2.38892 (7) obj. 624294 iterations 160
Cbc0038I Pass  38: suminf.    3.00000 (6) obj. 624254 iterations 144
Cbc0038I Pass  39: suminf.    3.00000 (6) obj. 624254 iterations 6
Cbc0038I Pass  40: suminf.    3.00000 (6) obj. 624254 iterations 6
Cbc0038I Pass  41: suminf.    2.38892 (7) obj. 624294 iterations 188
Cbc0038I Pass  42: suminf.    2.38892 (7) obj. 624294 iterations 34
Cbc0038I Pass  43: suminf.    3.00000 (6) obj. 6

Cbc0038I Full problem 256 rows 32640 columns, reduced to 44 rows 84 columns
Cbc0038I Mini branch and bound improved solution from 624429 to 624427 (3.38 seconds)
Cbc0038I Round again with cutoff of 624313
Cbc0038I Reduced cost fixing fixed 10616 variables on major pass 4
Cbc0038I Pass  33: suminf.    3.00000 (6) obj. 624213 iterations 22
Cbc0038I Pass  34: suminf.    3.00000 (6) obj. 624213 iterations 22
Cbc0038I Pass  35: suminf.    3.00000 (6) obj. 624213 iterations 7
Cbc0038I Pass  36: suminf.    4.31892 (34) obj. 624313 iterations 381
Cbc0038I Pass  37: suminf.    2.96220 (7) obj. 624313 iterations 497
Cbc0038I Pass  38: suminf.    2.96220 (7) obj. 624313 iterations 65
Cbc0038I Pass  39: suminf.    3.00000 (6) obj. 624311 iterations 257
Cbc0038I Pass  40: suminf.    3.00000 (9) obj. 624313 iterations 173
Cbc0038I Pass  41: suminf.    3.11094 (9) obj. 624313 iterations 140
Cbc0038I Pass  42: suminf.    2.96220 (7) obj. 624313 iterations 147
Cbc0038I Pass  43: suminf.    2.96220 (7) 

Cbc0038I Pass  31: suminf.    3.00000 (9) obj. 624297 iterations 128
Cbc0038I Pass  32: suminf.    3.00000 (6) obj. 624292 iterations 573
Cbc0038I Pass  33: suminf.    3.00000 (6) obj. 624292 iterations 244
Cbc0038I Pass  34: suminf.    6.68249 (29) obj. 624297 iterations 477
Cbc0038I Pass  35: suminf.    3.00000 (6) obj. 624285 iterations 401
Cbc0038I Pass  36: suminf.    3.00000 (6) obj. 624285 iterations 196
Cbc0038I No solution found this major pass
Cbc0038I Before mini branch and bound, 32459 integers at bound fixed and 0 continuous
Cbc0038I Full problem 256 rows 32640 columns, reduced to 46 rows 89 columns
Cbc0038I Mini branch and bound improved solution from 624352 to 624348 (3.95 seconds)
Cbc0038I Round again with cutoff of 624259
Cbc0038I Reduced cost fixing fixed 13188 variables on major pass 5
Cbc0038I Pass  36: suminf.    3.00000 (6) obj. 624217 iterations 0
Cbc0038I Pass  37: suminf.    3.00000 (6) obj. 624217 iterations 11
Cbc0038I Pass  38: suminf.    3.00000 (6) obj. 62

# Reconstructions, PSNRs

In [17]:
for Lambda in Lambdas:
# Better MSB 0 assignment ##################################### 
    pairs_show_file_name = image_name + '_pairs_' +str(Lambda)+'.npy' 
    mip_pairs_show = np.load(pairs_show_file_name)

    pairs_show = np.transpose(mip_pairs_show)
    abs_pairs_show = abs(pairs_show)
    pair_diff = abs(abs_pairs_show [:,0] - abs_pairs_show [:,1])
    pair_diff = pair_diff[..., None]
    pairs_diff = np.append(pairs_show, pair_diff, axis = 1)
    pairs_diff_sort = pairs_diff[pairs_diff[:,2].argsort()[::-1]]

    msb0_elements = np.zeros(np.shape(mip_pairs_show)[1])
    msb1_elements = np.zeros(np.shape(mip_pairs_show)[1])

    opt_pairs_prbs = []
    opt_pairs_prbs_dic = {}

    ac_dists = []
    for i in range(0,np.shape(mip_pairs_show)[1]):
        pair1 = pairs_diff_sort[i][0]
        pos1 = np.where(ac_values_n0 == pair1)
        prb_pair1 = prbs[pos1]

        pair2 = pairs_diff_sort[i][1]
        pos2 = np.where(ac_values_n0 == pair2)
        prb_pair2 = prbs[pos2]

        if not np.mod(i,2):
            if abs(pair1) < abs(pair2):
                msb0_elements [i] = pair1
                msb1_elements [i] = pair2
            else:
                msb0_elements [i] = pair2
                msb1_elements [i] = pair1
        if np.mod(i,2):
            if abs(pair1) > abs(pair2):
                msb0_elements [i] = pair1
                msb1_elements [i] = pair2
            else:
                msb0_elements [i] = pair2
                msb1_elements [i] = pair1   


        # probability of pairs
        pair_prb = prb_pair1 + prb_pair2
        opt_pairs_prbs = np.append(opt_pairs_prbs, pair_prb)
        element1 = pair1
        opt_pairs_prbs_dic[element1] = pair_prb

        # distortion between ac pairs
        ac_dist = 0.5*(pair1-pair2)**2*pair_prb
        ac_dists = np.append(ac_dists, ac_dist)

    ac_dist_total = sum(ac_dists)
    ac_dist_total_db = 10*np.log10(ac_dist_total)
    ac_dist_total_dbs = np.append(ac_dist_total_dbs, ac_dist_total_db)


    #     ###### For Lambdas
    opt_pairs_prbs_lambda_dic [Lambda] = opt_pairs_prbs_dic


    # Reconstruction
    eve_msb0_intensities = np.zeros([height,width])
    eve_msb1_intensities = np.zeros([height,width])
    for i in range(0,int(height/8)):
        for j in range(0,int(width/8)):
            blk = imf[8*i:8*(i+1), 8*j:8*(j+1)]
            shifted_blk = blk - 128 #level_shift for dct
            dct = cv2.dct(shifted_blk)
            nq_coef = np.round(dct)
            nq_coef_ac = np.delete(nq_coef, [0,0])
            nq_coef_dc = nq_coef[0,0]

            eve_ac_msb0 = nq_coef_ac.copy()
            eve_ac_msb1 = nq_coef_ac.copy()
            for m in range(np.size(nq_coef_ac)):
                if nq_coef_ac[m] in msb0_elements:
                    eve_ac_msb0[m] =  nq_coef_ac[m]

                    pos = np.where(msb0_elements == nq_coef_ac[m])
    #                     poss = np.where(msb0_elements == nq_coef_ac[m])
    #                     pos = poss[0][0]

                    eve_ac_msb1[m] = msb1_elements[pos]
                if nq_coef_ac[m] in msb1_elements:
                    eve_ac_msb1[m] =  nq_coef_ac[m]

                    pos = np.where(msb1_elements == nq_coef_ac[m])
    #                     poss = np.where(msb1_elements == nq_coef_ac[m])
    #                     pos = poss[0][0]
                    eve_ac_msb0[m] = msb0_elements[pos]


            eve_msb0_coef1 = np.insert(eve_ac_msb0, 0, 128)
#             eve_msb0_coef1 = np.insert(eve_ac_msb0, 0, nq_coef_dc)
            eve_msb0_coef = np.reshape(eve_msb0_coef1,[8,8])

            eve_msb0_blk1 = cv2.idct(eve_msb0_coef)
            eve_msb0_blk2 = eve_msb0_blk1+128 # level_shift after idct
            eve_msb0_blk = np.round(eve_msb0_blk2)  
            eve_msb0_intensities [8*i:8*(i+1), 8*j:8*(j+1)]= eve_msb0_blk


            eve_msb1_coef1 = np.insert(eve_ac_msb1, 0, 128)
#             eve_msb1_coef1 = np.insert(eve_ac_msb1, 0, nq_coef_dc)
            eve_msb1_coef = np.reshape(eve_msb1_coef1,[8,8])

            eve_msb1_blk1 = cv2.idct(eve_msb1_coef)
            eve_msb1_blk2 = eve_msb1_blk1+128 # level_shift after idct
            eve_msb1_blk = np.round(eve_msb1_blk2)  
            eve_msb1_intensities [8*i:8*(i+1), 8*j:8*(j+1)]= eve_msb1_blk


    eve_msb0_modified = eve_msb0_intensities.copy()
    eve_msb0_modified[eve_msb0_modified<0] = 0
    eve_msb0_modified[eve_msb0_modified>255] = 255
    #eve_msb0_uint8 = eve_msb0_intensities.astype(np.uint8)
    filename = image_name +'_msb0_'+str(Lambda)+'.png'
    cv2.imwrite(filename, eve_msb0_modified)
    
    eve_msb1_modified = eve_msb1_intensities.copy()
    eve_msb1_modified[eve_msb1_modified<0] = 0
    eve_msb1_modified[eve_msb1_modified>255] = 255
#     eve_msb1_uint8 =eve_msb1_intensities.astype(np.uint8)
    filename = image_name +'_msb1_'+str(Lambda)+'.png'
    cv2.imwrite(filename, eve_msb1_modified)

    # Distortion
    eve_msb0_dist_db = 10*np.log10(eve_distortion(eve_msb0_modified))
    eve_msb1_dist_db = 10*np.log10(eve_distortion(eve_msb1_modified))

    psnr_msb0_db = 20*np.log10(255)-eve_msb0_dist_db
    psnr_msb1_db = 20*np.log10(255)-eve_msb1_dist_db

    avg_psnr = \
    20*np.log10(255)-10*np.log10((eve_distortion(eve_msb0_modified)
                                 +eve_distortion(eve_msb1_modified))/2)
    avg_psnrs = np.append(avg_psnrs, avg_psnr)
    psnr_msb0s = np.append(psnr_msb0s, psnr_msb0_db)
    psnr_msb1s = np.append(psnr_msb1s, psnr_msb1_db)

/tmp/ipykernel_98876/4134898774.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  feve_intensities = eve_intensities.astype(np.float)
/tmp/ipykernel_98876/4134898774.py:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  fintensities = intensities.astype(np.float)
